Make sure you update the **STUDENT_TOKEN**, and then run this block. You should use FIRSTNAME-SURNAME(S) as the TOKEN.

The code in this block can be safely ignored.

In [313]:
#Update your token
STUDENT_TOKEN = 'GABRIEL DE OLAGUIBEL'

## ignore this code, just used for submission
import requests
import pprint
import json

def get_problem(problem_id):
  r = requests.get('https://emarchiori.eu.pythonanywhere.com/get-problem?TOKEN=%s&problem=%s' % (STUDENT_TOKEN, problem_id))
  if not r.status_code == 200:
    print('\033[91m' + str(r.status_code))
  return r.json()

def submit_answer(problem_id, answer):
  r = requests.get('https://emarchiori.eu.pythonanywhere.com/submit-answer?TOKEN=%s&problem=%s' % (STUDENT_TOKEN, problem_id), json = answer)
  if not r.status_code == 200:
    print('\033[91m' + str(r.status_code))
  result = r.json()['result']
  if result == 'PASSED':
    print('\033[92m' + result)
  else:
    print('\033[91m' + result)


# This is a base class for the problems, you will need to create a version of
# this for each problem, but that makes reusing the search code easier.
class Problem:
  def is_end(self, state): # Checks if the given state is an end or goal state
    return False

  def get_initial_state(self): # Returns the initial state of the problem
    return None

  def expand(self, parent): # Returns the successor states of a given state (parent)
    return []

  def heuristic(self, state): # Returns a heuristic value for the given state. (0 in base class)
    return 0


### Game Implementation provided by the professor

In [314]:
# Sliding tiles problem (BFS)

class SlidingTiles(Problem): # Class to represent the sliding tiles problem
  def __init__(self, initial_state, final_state):
    self.initial_state = initial_state
    self.final_state = final_state

  # We only need to check against the final state to know we are at an end state
  def is_end(self, state): # overide the is_end method from the Problem class
    return state == self.final_state

  def get_initial_state(self):
    return self.initial_state

  # Expand will have, at most, 4 valid actions, were the empty position moves left,
  # right, up or down. We use '%' and '//' operations to check when this is valid
  # as we are using a list representation of a 3x3 matrix.
  def expand(self, parent): # overide the expand method from the Problem class
    successors = [] #
    state = parent['state'] # state is the current state
    new_cost = parent['cost'] + 1 # new_cost is the cost of the current state + 1 since we measure cost by the number of moves
    pos = state.index(0) # pos is the position of the hole/empty space

    # pos - 1 (move empty to the left)
    # This move is valid from the following positions (not the ones with -)
    # - 1 2
    # - 4 5
    # - 7 8
    if pos % 3 > 0: # if the position is not in the first column
      new_state = state.copy()
      new_state[pos], new_state[pos - 1] = new_state[pos - 1], new_state[pos] # swap the empty space with the tile to the left
      successors.append({'state': new_state, 'parent': parent, 'action': 'L', 'cost': new_cost}) # add the new state to the successors list

    # pos + 1 (move empty to the right)
    # This move is valid from the following positions (not the ones with -)
    # 0 1 -
    # 3 4 -
    # 6 7 -
    if pos % 3 < 2: # if the position is not in the last column
      new_state = state.copy()
      new_state[pos], new_state[pos + 1] = new_state[pos + 1], new_state[pos] # swap
      successors.append({'state': new_state, 'parent': parent, 'action': 'R', 'cost': new_cost})

    # pos - 3 (move empty up)
    # This move is valid from the following positions (not the ones with -)
    # - - -
    # 3 4 5
    # 6 7 8
    if pos // 3 > 0: # if the position is not in the first row
      new_state = state.copy()
      new_state[pos], new_state[pos - 3] = new_state[pos - 3], new_state[pos]
      successors.append({'state': new_state, 'parent': parent, 'action': 'U', 'cost': new_cost})

    # pos + 3 (move empty down)
    # This move is valid from the following positions (not the ones with -)
    # 0 1 2
    # 3 4 5
    # - - -
    if pos // 3 < 2:
      new_state = state.copy()
      new_state[pos], new_state[pos + 3] = new_state[pos + 3], new_state[pos]
      successors.append({'state': new_state, 'parent': parent, 'action': 'D', 'cost': new_cost})

    return successors



## Exercise 1: 
#### New implementations:
- **Report the number of Visited Nodes**: To keep track of the number of visited nodes, I simply add a counter that increments every time we visit a new node. In the BFS implementation, this would be every time we pull a node from the frontier and expand it.
- **Report the effective branching factor**: The effective branching factor for a tree-based search algorithm like BFS can be calculated as: 

b = Number of nodes Generated / Number of nodes visited. 

I keep track of the number of nodes expanded and generated in the same way I keep track of the number of visited nodes.


In [315]:
import queue

# We define the failure value clearly, to simplify checking if needed
FAILURE = {'state': None, 'parent': None, 'action': None, 'cost': None}

# Uodated BFS code
def bfs_modified(problem):
    initial_state = problem.get_initial_state()
    final_state = problem.final_state # Access the final state directly from the problem instance
    inital_node = {'state': initial_state, 'parent': None, 'action': None, 'cost': 0}

    # Check if the intial state is already the final state
    if initial_state == final_state:
        return inital_node, 1, 0 # return the initial node, 1 node visited and 0 effective branching factor

    frontier = queue.Queue() # initialize queue
    frontier.put(inital_node) # add the initial node to the frontier
    reached = [initial_state] # add the initial state to the reached list

    # Initialize counters for finding visited nodes and branching factor
    nodes_visited = 0 # nodes pulled from the frontier and expanded
    nodes_generated = 0 # nodes generated in the frontier

    while not frontier.empty():
        node = frontier.get() # get the first node in the frontier
        nodes_visited += 1 # increment the expanded node counter

        if problem.is_end(node['state']):
            effective_branching_factor = nodes_generated / nodes_visited 
            return node, nodes_visited, effective_branching_factor
        
        for successor in problem.expand(node): # expand the node
            if not successor['state'] in reached: # only add the node if it is not in the reached list
                reached.append(successor['state']) # add the state to the reached list
                frontier.put(successor) # add the node to the frontier
                nodes_generated += 1
    
    effective_branching_factor = nodes_generated / nodes_visited if nodes_visited > 0 else 0 # calculate the effective branching factor. Add a check to avoid division by zero
    return FAILURE, nodes_visited, effective_branching_factor # return the failure value if no solution is found, nodes expanded and the effective branching factor

In [358]:
def json_to_list(state): # function to convert the state from a string to a list
  return list(map(lambda n: int(n), state.split(',')))

def submit_solution_to_server(initial_state, path):
    # Convert the initial state to the required string format
    initial_state_str = ','.join(map(lambda n: str(n), initial_state))
    
    # If the initial state is already the final state, set the path to a default value (e.g., 'U')
    if not path:
        path = [''] # enter here the expected path for an already solved problem

    # Prepare the answer to be submitted to the server
    answer = {
        'initial_state': initial_state_str,
        'path': '-'.join(map(lambda n: n['action'], filter(lambda n: not n['action'] == None, path)))
    }
    
    # Submit the answer to the server for checking
    submit_result = submit_answer('sliding_tiles', answer)
    
    return submit_result

# Get the problem from the server
problem = get_problem('sliding_tiles')
pprint.pprint(problem)

# Convert the state from a string to a list
initial_state = json_to_list(problem['initial-state'])
final_state = json_to_list(problem['final-state'])

# Un-comment these lines to override the initial state provided by the server if needed for debugging
# initial_state = [0, 2, 3, 1, 4, 6, 7, 5, 8] # easy (~4 depth)
# initial_state = [1, 2, 5, 8, 7, 6, 4, 3, 0] # hard (~15 depth)
# initial_state = [3, 6, 2, 7, 0, 1, 8, 4, 5] # very hard (>20 depth)


# Use the modified BFS to solve the problem and get the nodes visited and branching factor
solution, nodes_visited, branching_factor = bfs_modified(SlidingTiles(initial_state, final_state))

# Display the nodes visited and the effective branching factor
print(f"Nodes Visited: {nodes_visited}")
print(f"Effective Branching Factor: {branching_factor:.2f}") 

# Reconstruct the path to the solution
path = []
node = solution
while node != None:
    path.insert(0, node)
    node = node['parent']

# Print the path in a way to make it easy to debug and check
for node in path:
    print('------- %s' % node['action'])
    state = node['state']
    for i in range(3):
        print('¦ %i %i %i ¦' % (state[i*3], state[i*3 + 1], state[i*3 + 2]))


submit_result = submit_solution_to_server(initial_state, path)
print(submit_result)
#print(path)


{'explanation': '3x3 grid of tiles, with one missing. Tiles can only move from '
                'an adjacent place to the empty one (0).',
 'final-state': '1,2,3,4,5,6,7,8,0',
 'initial-state': '1,0,3,5,2,6,4,7,8',
 'path-format': '[UDLR](-[UDLR])*'}
Nodes Visited: 52
Effective Branching Factor: 1.60
------- None
¦ 1 0 3 ¦
¦ 5 2 6 ¦
¦ 4 7 8 ¦
------- D
¦ 1 2 3 ¦
¦ 5 0 6 ¦
¦ 4 7 8 ¦
------- L
¦ 1 2 3 ¦
¦ 0 5 6 ¦
¦ 4 7 8 ¦
------- D
¦ 1 2 3 ¦
¦ 4 5 6 ¦
¦ 0 7 8 ¦
------- R
¦ 1 2 3 ¦
¦ 4 5 6 ¦
¦ 7 0 8 ¦
------- R
¦ 1 2 3 ¦
¦ 4 5 6 ¦
¦ 7 8 0 ¦
PASSED
None


## Exercise 2: 
#### A* implementation:
- **Solve sliding-tile puzzles with A* Search**: Implemented more than 1 heuristic: Manhattan distance and ...
- **Report the number of Visited Nodes**: To keep track of the number of visited nodes, I simply add a counter that increments every time we visit a new node. In the A* implementation, this would be every time we pull a node from the frontier and expand it.
- **Report the effective branching factor**: The effective branching factor for a tree-based search algorithm like A* can be calculated as: 

b = Number of nodes Generated / Number of nodes visited. 


In [320]:
# Define the heuristic functions

# Heuristic 1: Manhattan Distance (sum of the distances of each tile to its final position)
def manhattan_distance(state, final_state):
    distance = 0
    for i in range(len(state)): # iterate over the state
        if state[i] != 0: # ignore the empty space
            goal_x, goal_y = divmod(final_state.index(state[i]), 3) # get the goal position of the tile
            state_x, state_y = divmod(i, 3) # get the current position of the tile
            distance += abs(goal_x - state_x) + abs(goal_y - state_y) # calculate the distance between the current position and the goal position
    return distance
'''''
Test
test_state = [2,3,0,
              1,4,6,
              7,5,8]

goal_state = [1, 2, 3, 
              4, 5, 6, 
              7, 8, 0]

manhattan_distance(test_state, goal_state)
'''''

# Heuristic 2: Number of misplaced tiles

"''\nTest\ntest_state = [2,3,0,\n              1,4,6,\n              7,5,8]\n\ngoal_state = [1, 2, 3, \n              4, 5, 6, \n              7, 8, 0]\n\nmanhattan_distance(test_state, goal_state)\n"

In [318]:
# A* Implementation with Heuristic of choice
import heapq 

# We define the failure value clearly, to simplify checking if needed
FAILURE = {'state': None, 'parent': None, 'action': None, 'cost': None}

def a_star(problem, heuristic):
  start_node = {'state': problem.get_initial_state(), 'parent': None, 'action': None, 'cost': 0} 
  frontier = [(heuristic(start_node['state'], problem.final_state),0, start_node)] # initialize the frontier with the initial state
  reached = {tuple(start_node['state']): start_node} # initialize the reached list with the initial state

  nodes_visited = 0 # nodes pulled from the frontier and expanded

  while frontier:
    # Get the node with the lowest cost from the frontier
    _, _, current_node = heapq.heappop(frontier)
    nodes_visited += 1

    # Check if the current node is the goal state
    if problem.is_end(current_node['state']):
      effective_branching_factor = len(reached) / nodes_visited if nodes_visited > 1 else 0
      return current_node, nodes_visited, effective_branching_factor
    
    # Expand the current node
    for successor in problem.expand(current_node):
      cost_to_successor = current_node['cost'] + 1 # cost to the successor is the cost of the current node + 1
      heuristic_val = heuristic(successor['state'], problem.final_state)
      total_cost = cost_to_successor + heuristic_val

      # Check if the successor is not in the reached list or if it is but with a higher cost
      if tuple(successor['state']) not in reached or cost_to_successor < reached[tuple(successor['state'])]['cost']:
          successor['cost'] = cost_to_successor # update the cost of the successor
          successor['parent'] = current_node # update the parent of the successor
          reached[tuple(successor['state'])] = successor # add the successor to the reached list
          heapq.heappush(frontier, (total_cost, id(successor), successor)) # add the successor to the frontier, use id(successor) to avoid comparing the nodes

  effective_branching_factor = len(reached) / nodes_visited if nodes_visited > 1 else 0
  return FAILURE, nodes_visited, effective_branching_factor

  

In [398]:
# Get the problem from the server
problem = get_problem('sliding_tiles')
pprint.pprint(problem)

# Convert the state from a string to a list
initial_state = json_to_list(problem['initial-state'])
final_state = json_to_list(problem['final-state'])

# Uncomment these lines to override the initial state provided by the server if needed for debugging
# initial_state = [0, 2, 3, 1, 4, 6, 7, 5, 8] # easy (~4 depth)
# initial_state = [1, 2, 5, 8, 7, 6, 4, 3, 0] # hard (~15 depth)
# initial_state = [3, 6, 2, 7, 0, 1, 8, 4, 5] # very hard (>20 depth)

# Use the A* algorithm to solve the problem and get the nodes visited and branching factor
solution, nodes_visited, branching_factor = a_star(SlidingTiles(initial_state, final_state), manhattan_distance)

# Display the nodes visited and the effective branching factor
print(f"Nodes Visited: {nodes_visited}")
print(f"Effective Branching Factor: {branching_factor:.2f}")

# Reconstruct the path to the solution
path = []
node = solution
while node != None:
    path.insert(0, node)
    node = node['parent']

# Print the path in a way to make it easy to debug and check
for node in path:
    print('------- %s' % node['action'])
    state = node['state']
    for i in range(3):
        print('¦ %i %i %i ¦' % (state[i*3], state[i*3 + 1], state[i*3 + 2]))

# Submit the solution to the server and print the result
submit_result = submit_solution_to_server(initial_state, path)
print(submit_result)


{'explanation': '3x3 grid of tiles, with one missing. Tiles can only move from '
                'an adjacent place to the empty one (0).',
 'final-state': '1,2,3,4,5,6,7,8,0',
 'initial-state': '1,0,3,5,2,6,4,7,8',
 'path-format': '[UDLR](-[UDLR])*'}
Nodes Visited: 6
Effective Branching Factor: 2.00
------- None
¦ 1 0 3 ¦
¦ 5 2 6 ¦
¦ 4 7 8 ¦
------- D
¦ 1 2 3 ¦
¦ 5 0 6 ¦
¦ 4 7 8 ¦
------- L
¦ 1 2 3 ¦
¦ 0 5 6 ¦
¦ 4 7 8 ¦
------- D
¦ 1 2 3 ¦
¦ 4 5 6 ¦
¦ 0 7 8 ¦
------- R
¦ 1 2 3 ¦
¦ 4 5 6 ¦
¦ 7 0 8 ¦
------- R
¦ 1 2 3 ¦
¦ 4 5 6 ¦
¦ 7 8 0 ¦
PASSED
None
